# Analyzing your games

Here is what we used to see what was going on in our games. You might find it useful for looking at how your own games unfold. To customize it, just fork and edit. Then add your own json files and link to them.

TODO: I'll try to add the scraper we used to get games through the API. In the meantime, you can download json files from the submissions-view link and upload them as a dataset.

In [ ]:
%%javascript

// disable scrolled windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
! conda install hvplot -y  # makes nicer plots

In [ ]:
import json
from glob import glob

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
import holoviews as hv
hv.extension("bokeh")
from IPython.display import display, HTML

from kaggle_environments import make

pd.set_option('max_columns', 25)

The code below produces the following info:

 - replay of the match
 - beginning and ending halie maps
 - a series of graphs comparing each player during the match: halite score, total assets, ships, etc,
 - the same info organized by player (sorry, these are not so pretty)
 
 
 
I hope it provides understanding of your bot's performace. Good luck during the final week!

In [ ]:

def open_game(filename):
    with open(filename) as file:
        game = json.load(file)
    env = make('halite', configuration=game['configuration'], steps=game['steps'])
    env.render(mode="ipython", width=500, height=400, autoplay=False, step=399)
    return game



def plot_halite(game):
    halite_start = game['steps'][1][0]['observation']['halite']
    halite_end = game['steps'][399][0]['observation']['halite']
    halite_start_2d = np.reshape(halite_start, (21,21))
    halite_end_2d = np.reshape(halite_end, (21,21))
    sns.set()
    fig, axes = plt.subplots(ncols=2, figsize=(16, 6))
    ax1, ax2 = axes
    ax1.set_title("Starting Halite")
    ax2.set_title("Ending Halite")
    sns.heatmap(halite_start_2d.astype(np.int16), ax=ax1,square=True, cmap='viridis', fmt='d',
                    annot=True, annot_kws={"size": 8})
    sns.heatmap(halite_end_2d.astype(np.int16), ax=ax2,square=True, cmap='viridis', fmt='d',
                    annot=True, annot_kws={"size": 8})  
    plt.show()
    

def get_types(game):
    # board halite
    halite = np.zeros((400,441), dtype=np.int16)
    for step in np.arange(400):
        halite[step] = game['steps'][step][0]['observation']['halite']

    # player_halite
    cash = np.zeros((400,4), dtype=np.uint32)
    for step in np.arange(400):
        for player in np.arange(4):
            p_cash = game['steps'][step][0]['observation']['players'] \
                            [player][0]
            cash[step,player] = p_cash

    # yards: 10k, 20k, 30k, 40k at yard position
    yards = np.zeros((400,441), dtype=np.uint16)
    for step in np.arange(400):
        for player in np.arange(4):
            yard_dict = game['steps'][step][0]['observation']['players'] \
                            [player][1]
            for v in yard_dict.values():
                yards[step,v] = (player+1)*10_000

    # ships: first digit is player number, remainder is cargo
    ships = np.zeros((400,441), dtype=np.uint16)
    for step in np.arange(400):
        for player in np.arange(4):
                ship_dict = game['steps'][step][0]['observation']['players'] \
                            [player][2]
                for v in ship_dict.values():
                    num = (player+1)*10_000 + v[1]
                    ships[step, v[0]] = num

    assets_dict = {'halite': halite,
                   'cash': cash,
                   'yards': yards,
                   'ships': ships}
    return assets_dict


# filter arrays by player and return values
def p_mask_ships(a, player):
    """Player is from 0 to 3."""
    p = player+1
    mask = (a > p*10_000) & (a < p*10_000+9_999)
    return mask


def p_mask_yards(a, player):
    """Player is from 0 to 3."""
    p = player+1
    mask = (a == p*10_000)
    return mask


def make_dfs(assets_dict):
    assets_long = np.zeros((1600, 4), dtype=np.int32)
    for i in range(4):
        start = i*400
        assets_long[start:start+400, 0] = assets_dict['cash'][:, i]
        assets_long[start:start+400, 1] = np.where(
            p_mask_ships(assets_dict['ships'],i),
            np.mod(assets_dict['ships'],10000), 0).sum(axis=1)
        assets_long[start:start+400, 2] = np.where(
            p_mask_ships(assets_dict['ships'],i), 500, 0).sum(axis=1)
        assets_long[start:start+400, 3] = np.where(
            p_mask_yards(assets_dict['yards'],i), 500, 0).sum(axis=1)

    assets_df = pd.DataFrame(data=assets_long , columns=['player_halite',
                        'cargo', 'ship_value', 'yard_value'])
    assets_df = assets_df.assign(
            step = np.tile(np.arange(400), 4),
            player = np.repeat(np.arange(4), 400),
            total_assets = assets_df[['cargo',
                'ship_value', 'yard_value']].sum(axis=1),
            ship_count = assets_df.ship_value / 500,
            yard_count = assets_df.yard_value / 500,
            avg_cargo = assets_df.cargo/assets_df.ship_value/500
            )
    board_halite_df = pd.DataFrame({'step': np.arange(400),
                        'board_halite': assets_dict['halite'].sum(axis=1)})
    return board_halite_df, assets_df


def plot_steps(board_halite_df, assets_df):
    plot_opts = dict(width=320, height=220,
                     xlim=(0,400),
                     xticks=list(range(0,401,50)),
                      ylabel=""
                     )
    asset_list = []
    cmap = ['yellow', 'red', 'green', 'purple']
    cols = ['player_halite', 'total_assets', 'ship_count', 'cargo',
                'avg_cargo', 'yard_count']
    h = board_halite_df.hvplot.line(x='step', y='board_halite', color='black',
                                   **plot_opts)
    
    # Plot by asset type
    for col in cols:
        if col == 'total_assets':
            start = h
        else:
            start = pd.DataFrame([0]).hvplot.line(**plot_opts) # blank

        for i, (_, df) in enumerate(assets_df.groupby('player')):
            start *= df.hvplot.line(x='step', y=col, title=col,
                        color=cmap[i])
        asset_list.append(start)
    asset_layout = hv.Layout(asset_list).opts(shared_axes=False).cols(3)
    display(asset_layout)

    # Plot by player
    player_opts = dict(width=280, height=260,
                     xlim=(0,400),
                     xticks=list(range(0,401,50)),
                     yticks=list(range(0,12000,2000)), 
                     ylabel="",
                     legend='top_left'
                     )
    player_list = []
    cols = ['player_halite','yard_value', 'ship_value']
    for i, (p_name, df) in enumerate(assets_df.groupby('player')):
        player = pd.DataFrame([0]).hvplot.line(**player_opts) # blank
        for col in cols:
            player *= df.hvplot.line(x='step', y=col, title=str(p_name),
                                     cmap=['gray', 'blue', 'green'], **player_opts)
        player_list.append(player)    
    player_layout = hv.Layout(player_list).opts(shared_axes=True)
    display(player_layout)


In [ ]:
# main
filenames = glob('../input/halitetemp/*.json')
filenames.sort(reverse=True)
for f in filenames[:10]:
    print(f)
    game = open_game(f)
    plot_halite(game)
    asset_dict = get_types(game)
    board_halite_df, assets_df = make_dfs(asset_dict)
    plot_steps(board_halite_df, assets_df)

# Addendum: Fast distances and directions

As bots become more complex, they need to know locations and distances of all the elements - halite, ships, shipyards - for a number of scenarios. Here's one way to calculate fast distances and fast matrices in general.


### Standard calculation
A typical distance calculation looks like the one here, from https://www.kaggle.com/philculliton/test-agents. It's pretty fast and is well-written.

In [ ]:
def manhattan_distance_single(i1, i2, size=21):
    """Gets the distance in one dimension between two columns or 
    two rows, including wraparound."""
    iMin = min(i1, i2)
    iMax = max(i1, i2)
    return min(iMax - iMin, iMin + size - iMax)


def manhattan_distance(pos1, pos2, size=21):
    """Gets the Manhattan distance between two positions, i.e.,
    how many moves it would take a ship to move between them."""
    # E.g. for 17-size board, 0 and 17 are actually 1 apart
    dx = manhattan_distance_single(pos1 % size, pos2 % size)
    dy = manhattan_distance_single(pos1 // size, pos2 // size)
    return dx + dy

In [ ]:
%%time

for i in tqdm(range(1_000_000)):
    a = manhattan_distance(0, 220)
print(a)

That's a million calculations in ~3.5 seconds (+/- depending on the run). It sounds like a lot and more than my bots use. But if you start factoring in all the objects and where ships could be in 2-3-4 moves, the numbers start to add up. Plus you may have other calculations that need to finish before using up the 6 seconds per turn, like loading weights into a CNN.

### Fast distance calculation

You can speed things up by precomputing the distance matrix for all points and using it directly as a lookup table. The code below is used to make the matrix (list of lists actually). We have 30 seconds at startup so there's plenty of time to make this matrix, even with using nested loops. The code below with numpy arrays is much, much faster though and might be useful during the game if you need to compute other matrices.

In [ ]:
%%time 

# do this only once per game

def dist_1d(a1, a2):
    amin = np.fmin(a1, a2)
    amax = np.fmax(a1, a2)
    adiff = amax-amin
    adist = np.fmin(adiff, 21-adiff)
    return adist


def make_dist_matrix():
    base = np.arange(21**2)
    idx1 = np.repeat(base, 21**2)
    idx2 = np.tile(base, 21**2)

    rowdist = dist_1d(idx1 // 21, idx2 // 21)
    coldist = dist_1d(idx1 % 21, idx2 % 21)

    dist_matrix = (rowdist + coldist).reshape(21**2, -1)
    return dist_matrix

distance_list = make_dist_matrix().tolist()

In [ ]:
# just for display purposes
pd.set_option('display.min_rows', 20)
display(pd.DataFrame(make_dist_matrix()))

![](http://)Here's the same test as with the original function except now we use the precalculated matrix. It takes ess than one second!

In [ ]:
%%time
for i in tqdm(range(1_000_000)):
    a = distance_list[0][220]
print(a)

### Fast directions
Here's a similar way to precalculate directions between any two points. You can slice the 3d array by the first two axes to get the pair of directions leading from one point to the other.

In [ ]:
def make_direction_matrix():
    """Used to get directions between any two points. Each value of the
    square matrix is a 1d-array of two directions."""

    base = np.arange(21**2)
    idx1 = np.repeat(base, 21**2)
    idx2 = np.tile(base, 21**2)

    row_diff = idx1//21 - idx2//21
    row_mask = ((row_diff>21/2) | ((row_diff>(-21/2))&(row_diff<0)))

    row_dir = np.empty((21**4), dtype="object")
    row_dir = np.where(row_mask, "SOUTH", "NORTH")
    row_dir[row_diff==0] = "no"

    col_diff = idx1 % 21 - idx2 % 21
    col_mask = ((col_diff>21/2) | ((col_diff>(-21/2))&(col_diff<0)))

    col_dir = np.empty((21**4), dtype="object")
    col_dir = np.where(col_mask, "EAST", "WEST")
    col_dir[col_diff==0] = "no"

    dir_matrix = np.stack((row_dir, col_dir), axis=1).reshape(21**2, 21**2, 2)
    return dir_matrix.astype('object')



Some examples:

In [ ]:
dir_matrix = make_direction_matrix()

print(dir_matrix[2, 47],
      dir_matrix[2, 20],  # wrapararound
      dir_matrix[2, 44],
      dir_matrix[2, 2]    # you're there!
      )